In [2]:
import yfinance as yf
import pandas as pd

# Load the CSV file containing the tickers
csv_file_path = 'filtered_stocks_under_30.csv'  # Adjust the path if necessary
tickers_df = pd.read_csv(csv_file_path)

# Assuming the column with tickers is named 'Ticker'
tickers_list = tickers_df['Ticker'].tolist()

# Check for any missing tickers
tickers_list = [ticker for ticker in tickers_list if pd.notnull(ticker)]

# Fetch current price data from Yahoo Finance using the tickers
current_price_data = yf.download(tickers_list, period='1d')['Adj Close']

# Display the current prices
print(current_price_data)

[*********************100%***********************]  1225 of 1225 completed


Ticker                     AADI   AAME       AAOI   ABAT  ABEO  ABOS   ABSI  \
Date                                                                          
2024-10-14 00:00:00+00:00  2.14  1.667  18.719999  0.995  6.27  2.36  4.116   

Ticker                     ABUS    ACCD    ACCO  ...  XPER  YEXT   YMAB  ZDGE  \
Date                                             ...                            
2024-10-14 00:00:00+00:00  3.89  3.6997  5.1735  ...  8.87  6.96  14.75  3.41   

Ticker                      ZIMV       ZUMZ  ZURA   ZVRA   ZYME  ZYXI  
Date                                                                   
2024-10-14 00:00:00+00:00  15.81  22.200001  4.12  8.005  13.34  7.93  

[1 rows x 1225 columns]


In [3]:
# Define a function to get fundamental data
def get_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Extract relevant fundamental data
    pe_ratio = info.get('trailingPE', None)
    earnings_growth = info.get('earningsQuarterlyGrowth', None)
    roe = info.get('returnOnEquity', None)
    rev_growth = info.get('revenueGrowth')
    profit_margin = info.get('profitMargins', None)
    debt_to_equity = info.get('debtToEquity', None)
    
    return {
        'Ticker': ticker,
        'Rev Growth': rev_growth,
        'PE_Ratio': pe_ratio,
        'Earnings_Growth': earnings_growth,
        'ROE': roe,
        'Profit Margin': profit_margin,
        'Debt_to_Equity': debt_to_equity
    }

In [4]:
# Fetch data for all Dow 30 stocks
fundamental_data = [get_fundamental_data(ticker) for ticker in current_price_data]
fundamental_df = pd.DataFrame(fundamental_data)

# Display the fundamental data
fundamental_df

,Ticker,Rev Growth,PE_Ratio,Earnings_Growth,ROE,Profit Margin,Debt_to_Equity
0,AADI,-0.004,None,NaN,-0.62264,-2.74774,1.340
1,AAME,-0.031,None,NaN,-0.03087,-0.01694,39.852
2,AAOI,0.040,None,NaN,-0.41227,-0.34838,58.580
3,ABAT,NaN,None,NaN,-0.85831,NaN,10.577
4,ABEO,NaN,None,NaN,-1.27380,NaN,31.628
...,...,...,...,...,...,...,...
1220,ZUMZ,0.046,None,-0.107,-0.15129,-0.06023,57.202
1221,ZURA,NaN,7.219298,NaN,-0.26440,NaN,NaN
1222,ZVRA,-0.475,None,NaN,-1.32319,-2.82013,183.102
1223,ZYME,1.748,None,NaN,-0.26315,-1.79424,4.946


In [5]:
fundamental_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ticker           1225 non-null   object 
 1   Rev Growth       1014 non-null   float64
 2   PE_Ratio         418 non-null    object 
 3   Earnings_Growth  343 non-null    float64
 4   ROE              1098 non-null   float64
 5   Profit Margin    929 non-null    float64
 6   Debt_to_Equity   928 non-null    float64
dtypes: float64(5), object(2)
memory usage: 67.1+ KB


In [6]:
fundamental_df.describe()

,Rev Growth,Earnings_Growth,ROE,Profit Margin,Debt_to_Equity
count,1014.000000,343.000000,1098.000000,929.000000,928.000000
mean,0.535385,0.824082,-0.535548,0.968639,169.926565
std,5.183832,4.375980,1.750838,33.481663,917.481572
min,-0.998000,-0.998000,-29.865520,-2.906630,0.008000
25%,-0.097000,-0.381500,-0.583937,-0.213770,6.554750
50%,0.013500,-0.111000,-0.123830,-0.018210,31.104500
75%,0.175500,0.331000,0.055995,0.078800,101.909750
max,118.308000,41.889000,2.808370,1019.742200,20355.047000


In [7]:
def filter_stocks(df):
    # Apply screening criteria
    filtered_df = df[
        (df['Earnings_Growth'] > 0.331000) &
        (df['Rev Growth'] > 0.013500) &
        (df['Profit Margin'] > 0.10) &
        (df['ROE'] > 0.055995) &
        (df['Debt_to_Equity'] < 31.104500)
    ]
    return filtered_df

# Apply the filter
filtered_stocks = filter_stocks(fundamental_df)

# Display the filtered stocks
print("Filtered Stocks:")
print(filtered_stocks)

Filtered Stocks:
     Ticker  Rev Growth   PE_Ratio  Earnings_Growth      ROE  Profit Margin  \
105     ASC       0.320   4.948571            1.558  0.25116        0.36314   
136     AXR       0.855  16.602272            2.019  0.08023        0.15635   
243    CHCI       0.199   14.12747            0.992  0.24251        0.17945   
369     EGY       0.069      7.555            3.169  0.18185        0.17823   
458    GAMB       0.176  14.716417           23.928  0.22404        0.22132   
541     IDR       0.893     37.025            5.583  0.21339        0.26689   
691    MDXG       0.073  10.026317           13.688  0.70047        0.26111   
827    OPXS       0.263  13.786884            1.189  0.26959        0.12826   
936    RDVT       0.298  23.705357            0.900  0.19352        0.23488   
1051   SRTS       1.557       16.0            4.224  0.13436        0.18273   

      Debt_to_Equity  
105            8.175  
136            0.028  
243           17.124  
369           17.566 

In [8]:
def add_sector_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    sector = info.get('sector', 'Unknown')
    return sector

# Add sector data to the filtered stocks
filtered_stocks['Sector'] = filtered_stocks['Ticker'].apply(add_sector_data)

# Group by Sector
sector_counts = filtered_stocks['Sector'].value_counts()
print("Sector Analysis:")
print(sector_counts)

Sector Analysis:
Sector
Industrials          2
Real Estate          2
Healthcare           2
Energy               1
Consumer Cyclical    1
Basic Materials      1
Technology           1
Name: count, dtype: int64


C:\Users\nilee\AppData\Local\Temp\ipykernel_20124\1538445913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_stocks['Sector'] = filtered_stocks['Ticker'].apply(add_sector_data)


In [9]:
filtered_stocks

,Ticker,Rev Growth,PE_Ratio,Earnings_Growth,ROE,Profit Margin,Debt_to_Equity,Sector
105,ASC,0.320,4.948571,1.558,0.25116,0.36314,8.175,Industrials
136,AXR,0.855,16.602272,2.019,0.08023,0.15635,0.028,Real Estate
243,CHCI,0.199,14.12747,0.992,0.24251,0.17945,17.124,Real Estate
369,EGY,0.069,7.555,3.169,0.18185,0.17823,17.566,Energy
458,GAMB,0.176,14.716417,23.928,0.22404,0.22132,18.497,Consumer Cyclical
541,IDR,0.893,37.025,5.583,0.21339,0.26689,9.705,Basic Materials
691,MDXG,0.073,10.026317,13.688,0.70047,0.26111,11.455,Healthcare
827,OPXS,0.263,13.786884,1.189,0.26959,0.12826,19.897,Industrials
936,RDVT,0.298,23.705357,0.900,0.19352,0.23488,2.607,Technology
1051,SRTS,1.557,16.0,4.224,0.13436,0.18273,0.865,Healthcare


In [14]:
filtered_stocks.to_csv('pennies.csv', index=False)